# Read news headline data for "AAPL"

In [1]:
import pandas as pd
apple_headlines_data=pd.read_csv('news_headline_sentiments_aapl.csv',parse_dates=True)
apple_headlines_data.head(5)

,Unnamed: 0,Unnamed: 0.1,news_headline,time_stamp,URL,source_id,sentiment_class,sentiment_scores
0,29,2012757,apple will refund at least $32.5m in app case,2014-01-27 10:17:47-08:00,//www.suntimes.com/business/24974362-420/apple...,1303,0,0.0
1,169,2012707,"icahn raises apple stake, now owns $3b in stock",2014-01-27 10:17:47-08:00,//www.suntimes.com/business/25119767-420/icahn...,1303,0,0.0
2,616,2012614,apple: people spent $10b in its app store in 2013,2014-01-27 10:17:47-08:00,//www.suntimes.com/business/24816093-420/apple...,1303,0,0.0
3,618,2007933,"apple s mac still influencing computing, 30 ye...",2014-01-27 10:17:47-08:00,http://www.nbcnews.com/business/apples-mac-sti...,1288,0,0.0
4,726,2011678,new apple tv set-top box expected in first hal...,2014-01-27 10:17:47-08:00,/business/technology/la-fi-tn-new-apple-tv-fir...,1298,0,0.0


# Get the required column

In [2]:
# Get the required columns used in the strategy
data=apple_headlines_data.loc[:,['time_stamp','news_headline','sentiment_class','sentiment_scores']]

#convert time column in date-time format
data['time_stamp']=pd.to_datetime(data['time_stamp'],utc=True)

# Sort the dataframe by datetime
data.sort_values(by='time_stamp',inplace=True)
data.head()

,time_stamp,news_headline,sentiment_class,sentiment_scores
0,2014-01-27 18:17:47+00:00,apple will refund at least $32.5m in app case,0,0.0
1,2014-01-27 18:17:47+00:00,"icahn raises apple stake, now owns $3b in stock",0,0.0
2,2014-01-27 18:17:47+00:00,apple: people spent $10b in its app store in 2013,0,0.0
3,2014-01-27 18:17:47+00:00,"apple s mac still influencing computing, 30 ye...",0,0.0
4,2014-01-27 18:17:47+00:00,new apple tv set-top box expected in first hal...,0,0.0


## Calculate trading time for each news headline

We will fetch all the headlines that were made between the previous day's close and next day's open.

To do this, we will use a function called get_trade_open which will give you the market open time when a particular headline should be used. In the function, we have followed following steps.
1. Calculate the current day opening time
2. Calculate the current day closing time
3. Calculate the previous day closing time
4. Calculate next day opening time
4. If the headline was made after the previous day's market close and before today's market open, then this column will contain the today's market open date and time.
5. Similarly, if the headline was made after the current day's market close and before next day's market open, then this column will contain the next day's market open date and time.

Note: News headline during the market hours will be ignored and will not be considered in sentiment calculation.


We will use the pandas function BDay to determine the next and previous business/trading day's dates.

In [3]:
#import datetime
from datetime import datetime,timedelta

#import BDay yo determine business day's dates
from pandas.tseries.offsets import BDay

#fetch the first date of the headline
date=data.time_stamp.iloc[0]
date

Timestamp('2014-01-27 18:17:47+0000', tz='UTC')

In [5]:
def get_trade_open(date):
    
    # If the headline was made on a weekend then subtract BDay(0) to fetch the next business day's market open time
    #pd.to_datetime(date).floor('d'), which set all the value is 00:00:00:
    curr_date_open = pd.to_datetime(date).floor('d').replace(hour=9,minute=30) - BDay(0)
    
    # If the tweet was made on a weekend then subtract  BDay(0) to fetch the next business day's market close time
    curr_date_close = pd.to_datetime(date).floor('d').replace(hour=16,minute=0) - BDay(0)
    
    # Calculate the previous business day's market close time for the headline 
    prev_date_close = (curr_date_open - BDay()).replace(hour=16,minute=0)
    
    # Calculate the next business day's market open time for the headline 
    next_date_open = (curr_date_close + BDay()).replace(hour=9,minute=30)
    
    # If the tweet was made after the close of the previous business day and
    # on the next day before the market opens then the function assigns the curr_date_open as the opening time
    # when this tweet should be used to trade
    if ((pd.to_datetime(date)>=prev_date_close) & (pd.to_datetime(date)<curr_date_open)):
        return curr_date_open
    
    # If the tweet was made after the close of the current business day and
    # on the next day open then the function assigns the next_date_open as the opening time
    # when this tweet should be used to trade
    elif ((pd.to_datetime(date)>=curr_date_close) & (pd.to_datetime(date)<next_date_open)):
        return next_date_open
    
    else:
        return None
    

In [6]:
# Apply the above function to get the trading time for each news headline
data["trading_time"] = data["time_stamp"].apply(get_trade_open)

In [7]:
data.tail()

,time_stamp,news_headline,sentiment_class,sentiment_scores,trading_time
15302,2016-05-30 01:15:01+00:00,commerce ministry favours local sourcing waive...,1,0.4215,2016-05-30 09:30:00+00:00
15310,2016-05-30 01:45:01+00:00,commerce ministry turns down apple?s proposal ...,-1,-0.1880,2016-05-30 09:30:00+00:00
15311,2016-05-30 01:45:01+00:00,nirmala sitharaman may help pave way for apple...,1,0.4019,2016-05-30 09:30:00+00:00
15309,2016-05-30 01:45:01+00:00,apple?s refurbished iphone plan: govt not in f...,-1,-0.4952,2016-05-30 09:30:00+00:00
15312,2016-05-30 02:15:02+00:00,discussing apple's request for local sourcing ...,0,0.0000,2016-05-30 09:30:00+00:00


## Calculate daily sentiment score
For all the news headlines with same trading time, we will compute the mean of its sentiment class. This will be the daily sentiment score. This is one of the approaches we have followed. 


In [8]:
apple_daily_sentiment = data.groupby('trading_time').sentiment_class.agg('mean').to_frame('score')

In [9]:
apple_daily_sentiment.head()

,score
trading_time,
2014-01-28 09:30:00+00:00,0.034483
2014-01-29 09:30:00+00:00,0.238095
2014-01-30 09:30:00+00:00,0.571429
2014-02-03 09:30:00+00:00,-0.200000
2014-02-04 09:30:00+00:00,-0.500000


In [10]:
# Reset the index
apple_daily_sentiment.reset_index(inplace=True)

In [11]:
apple_daily_sentiment.head(2)

,trading_time,score
0,2014-01-28 09:30:00+00:00,0.034483
1,2014-01-29 09:30:00+00:00,0.238095


In [12]:
# Create a column and store only dates of the trading_time for the better visualisation of the dates
apple_daily_sentiment['Date'] = pd.to_datetime(
    apple_daily_sentiment['trading_time']).dt.floor('d')

In [13]:
apple_daily_sentiment.head(2)

,trading_time,score,Date
0,2014-01-28 09:30:00+00:00,0.034483,2014-01-28 00:00:00+00:00
1,2014-01-29 09:30:00+00:00,0.238095,2014-01-29 00:00:00+00:00


In [14]:
# Remove the time zone information from the time column
apple_daily_sentiment['Date'] = apple_daily_sentiment['Date'].apply(lambda d: d.replace(tzinfo=None))

In [15]:
# Set the index
apple_daily_sentiment.set_index("Date", inplace=True)
apple_daily_sentiment.head()

,trading_time,score
Date,,
2014-01-28,2014-01-28 09:30:00+00:00,0.034483
2014-01-29,2014-01-29 09:30:00+00:00,0.238095
2014-01-30,2014-01-30 09:30:00+00:00,0.571429
2014-02-03,2014-02-03 09:30:00+00:00,-0.200000
2014-02-04,2014-02-04 09:30:00+00:00,-0.500000


## Store daily sentiment score in CSV file 

In [18]:
apple_daily_sentiment.to_csv('apple_daily_sentiment2.csv')